In [ ]:
import os

# 1. Extract Keypoints
load video data and extract keypoints using mediapose into /data/pose_data.csv

In [ ]:
from src import extract_keypoints as extract

# Process all MP4s from a folder and extract labeled keypoints
video_dir = os.path.join("..", "data", "raw")
df = extract.process_directory(video_dir)

# Save result for training
csv_path = os.path.join("..", "data", "pose_data.csv")
extract.save_dataframe(df, csv_path)


# 2. Train Model
create classifier for the keypoints and save the model in models/tf_model

In [ ]:
from src import train_pose_model as train
from sklearn.model_selection import train_test_split

model_dir = os.path.join("..", "models", "tf_model")
label_map_path = os.path.join("..", "models", "label_map.json")

# Load data
X, y = train.load_pose_data(csv_path)
y_encoded, label_encoder = train.encode_labels(y)

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Build and train model
model = train.build_model(input_dim=X.shape[1], num_classes=len(label_encoder.classes_))
model.summary()
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=32)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

# Save model and labels
train.save_model(model, model_dir)
train.save_label_map(label_encoder, label_map_path)